In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install zeugma
!pip install deslib

  Created wheel for zeugma: filename=zeugma-0.48-cp37-none-any.whl size=8778 sha256=37ecac55d2a55dba93783831e4f8d38d38b797a3fa8330b08b8205c9f0427a69
  Stored in directory: /root/.cache/pip/wheels/6a/b5/bc/5183ac478b0071d04d3ed0c0dd4a43db94c5c8ffb317b5eb53
Successfully built zeugma
     |████████████████████████████████| 163kB 6.6MB/s 


In [4]:
from zeugma.embeddings import EmbeddingTransformer

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [6]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from preprocessing import *
from function import *

In [8]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from deslib.util.aggregation import *

In [9]:
w2v = EmbeddingTransformer('word2vec')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [10]:
glove = EmbeddingTransformer('glove')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [11]:
fasttext = EmbeddingTransformer('fasttext')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [13]:
uri_train = "/content/drive/MyDrive/Dataset/Folds/F1_train_zw.csv"
uri_val = "/content/drive/MyDrive/Dataset/Folds/F1_train_zw.csv"
uri_test = "/content/drive/MyDrive/Dataset/Folds/F1_test_zw.csv"

df_train = pd.read_table(uri_train, sep=',')
df_val   = pd.read_table(uri_val, sep=',')
df_test = pd.read_table(uri_test, sep=',')

In [14]:
train = df_train['text'].fillna(' ').apply(pre_processing)
val = df_val['text'].fillna(' ').apply(pre_processing)
test, class_test = df_test['text'].fillna(' ').apply(pre_processing), df_test['norm']

In [15]:
cv = CountVectorizer(analyzer='word', lowercase=True, stop_words='english')
cv.fit_transform(train.values.astype('U'))
     
tfidf =  TfidfVectorizer(analyzer='word', lowercase=True, use_idf=True, stop_words='english')
tfidf.fit_transform(train.values.astype('U'))

<13525x13864 sparse matrix of type '<class 'numpy.float64'>'
	with 90779 stored elements in Compressed Sparse Row format>

In [16]:
svm = {
    'CV': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=0.1, probability=True),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=1, probability=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT': fasttext,
    }
}

In [17]:
svm_clfs = {

}
for ext, clf in svm.items():
    svm_clfs[ext] = get_classifier(clf['CLF'], train, df_train['norm'], clf['EXT'])

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [18]:
df_pred_train = pd.DataFrame(df_train['norm'])
df_pred_val = pd.DataFrame(df_val['norm'])
df_pred_test = pd.DataFrame(df_test['norm'])

df_prob_val = pd.DataFrame(df_val['norm'])
df_prob_train = pd.DataFrame(df_train['norm'])
df_prob_test = pd.DataFrame(df_test['norm'])
for ext, clf in svm_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["SVM-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["SVM-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["SVM-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "SVM-{}-{}".format(ext,clf.classes_[0]), 
    "SVM-{}-{}".format(ext,clf.classes_[1]), 
    "SVM-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [19]:
lr = {
    'CV': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': fasttext,
    }
}

In [20]:
lr_clfs = {

}
for ext, clf in lr.items():
    lr_clfs[ext] = get_classifier(clf['CLF'], train, df_train['norm'], clf['EXT'])

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

In [21]:
for ext, clf in lr_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["LR-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["LR-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["LR-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "LR-{}-{}".format(ext,clf.classes_[0]), 
    "LR-{}-{}".format(ext,clf.classes_[1]), 
    "LR-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [22]:
rf = {
    'CV': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=20),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=20),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': fasttext,
    }
}

In [23]:
rf_clfs = {

}
for ext, clf in rf.items():
    rf_clfs[ext] = get_classifier(clf['CLF'], train, df_train['norm'], clf['EXT'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 20
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
building tree 2 of 20
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
building tree 3 of 20
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
building tree 4 of 20
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
building tree 5 of 20
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.2s remaining:    0.0s
building tree 6 of 20
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.4s remaining:    0.0s
building tree 7 of 20
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    1.6s remaining:    0.0s
building tree 8 of 20
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    1.8s remaining:    0.0s
building tree 9 of 20
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.1s remaining:    0.0s
b

In [24]:
for ext, clf in rf_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["RF-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["RF-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["RF-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "RF-{}-{}".format(ext,clf.classes_[0]), 
    "RF-{}-{}".format(ext,clf.classes_[1]), 
    "RF-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [25]:
nb = {
    'CV': {
        'CLF': MultinomialNB(alpha=1, fit_prior=True),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MultinomialNB(alpha=0.1, fit_prior=True),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': BernoulliNB(alpha=0.1, fit_prior=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': BernoulliNB(alpha=0.1, fit_prior=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': BernoulliNB(alpha=1, fit_prior=True),
        'EXT': fasttext,
    }
}

In [26]:
nb_clfs = {

}
for ext, clf in nb.items():
    nb_clfs[ext] = get_classifier(clf['CLF'], train, df_train['norm'], clf['EXT'])

In [27]:
for ext, clf in nb_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["NB-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["NB-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["NB-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "NB-{}-{}".format(ext,clf.classes_[0]), 
    "NB-{}-{}".format(ext,clf.classes_[1]), 
    "NB-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [28]:
mlp = {
    'CV': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='lbfgs'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='lbfgs'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': fasttext,
    }
}

In [29]:
mlp_clfs = {

}
for ext, clf in mlp.items():
    mlp_clfs[ext] = get_classifier(clf['CLF'], train, df_train['norm'], clf['EXT'])

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Iteration 1, loss = 0.63829315
Iteration 2, loss = 0.53182532
Iteration 3, loss = 0.50969961
Iteration 4, loss = 0.49251546
Iteration 5, loss = 0.47916379
Iteration 6, loss = 0.46670504
Iteration 7, loss = 0.45322165
Iteration 8, loss = 0.44080194
Iteration 9, loss = 0.42988291
Iteration 10, loss = 0.41730281
Iteration 11, loss = 0.40580664
Iteration 12, loss = 0.39613690
Iteration 13, loss = 0.38471277
Iteration 14, loss = 0.37467684
Iteration 15, loss = 0.36410800
Iteration 16, loss = 0.35388373
Iteration 17, loss = 0.34293330
Iteration 18, loss = 0.33397674
Iteration 19, loss = 0.32530846
Iteration 20, loss = 0.31727006


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.73825671
Iteration 2, loss = 0.67742141
Iteration 3, loss = 0.66891187
Iteration 4, loss = 0.66276536
Iteration 5, loss = 0.65747072
Iteration 6, loss = 0.65389524
Iteration 7, loss = 0.64948711
Iteration 8, loss = 0.64541832
Iteration 9, loss = 0.64284410
Iteration 10, loss = 0.63907153
Iteration 11, loss = 0.63708087
Iteration 12, loss = 0.63423387
Iteration 13, loss = 0.63089318
Iteration 14, loss = 0.62910678
Iteration 15, loss = 0.62695661
Iteration 16, loss = 0.62475149
Iteration 17, loss = 0.62201913
Iteration 18, loss = 0.62113453
Iteration 19, loss = 0.62020382
Iteration 20, loss = 0.61751096


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.69767096
Iteration 2, loss = 0.55214084
Iteration 3, loss = 0.52388896
Iteration 4, loss = 0.50698711
Iteration 5, loss = 0.49699341
Iteration 6, loss = 0.48981973
Iteration 7, loss = 0.48174166
Iteration 8, loss = 0.47391871
Iteration 9, loss = 0.46987047
Iteration 10, loss = 0.46240108
Iteration 11, loss = 0.45563331
Iteration 12, loss = 0.45151517
Iteration 13, loss = 0.44531528
Iteration 14, loss = 0.44129969
Iteration 15, loss = 0.43656418
Iteration 16, loss = 0.43022343
Iteration 17, loss = 0.42513897
Iteration 18, loss = 0.42006554
Iteration 19, loss = 0.41689030
Iteration 20, loss = 0.41234346


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [30]:
for ext, clf in mlp_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["MLP-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["MLP-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["MLP-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "MLP-{}-{}".format(ext,clf.classes_[0]), 
    "MLP-{}-{}".format(ext,clf.classes_[1]), 
    "MLP-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [31]:
extra = {
    'CV': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=20),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=20),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': fasttext,
    }
}

In [32]:
extra_clfs = {

}
for ext, clf in extra.items():
    extra_clfs[ext] = get_classifier(clf['CLF'], train, df_train['norm'], clf['EXT'])


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 20
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
building tree 2 of 20
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s
building tree 3 of 20
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.1s remaining:    0.0s
building tree 4 of 20
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.6s remaining:    0.0s
building tree 5 of 20
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.0s remaining:    0.0s
building tree 6 of 20
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    2.4s remaining:    0.0s
building tree 7 of 20
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    2.8s remaining:    0.0s
building tree 8 of 20
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    3.2s remaining:    0.0s
building tree 9 of 20
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.6s remaining:    0.0s
b

In [33]:
for ext, clf in extra_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["EXTRA-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["EXTRA-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["EXTRA-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "EXTRA-{}-{}".format(ext,clf.classes_[0]), 
    "EXTRA-{}-{}".format(ext,clf.classes_[1]), 
    "EXTRA-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [34]:
knn = {
    'CV': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': KNeighborsClassifier(n_neighbors=3, algorithm='auto'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': fasttext,
    }
}

In [35]:
knn_clfs = { }
for ext, clf in knn.items():
    knn_clfs[ext] = get_classifier(clf['CLF'], train, df_train['norm'], clf['EXT'])

In [36]:
for ext, clf in knn_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["KNN-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["KNN-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["KNN-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "KNN-{}-{}".format(ext,clf.classes_[0]), 
    "KNN-{}-{}".format(ext,clf.classes_[1]), 
    "KNN-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


# CNN

In [37]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=300

y_train = to_categorical(df_train['norm'])
y_val = to_categorical(df_val['norm'])

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train)

seq_train = tokenizer.texts_to_sequences(train)
seq_val = tokenizer.texts_to_sequences(val)
seq_test = tokenizer.texts_to_sequences(test)

data_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
data_val = pad_sequences(seq_val, maxlen=MAX_SEQUENCE_LENGTH)
data_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [38]:
cnn_cv = get_CNN(cv, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid')
cnn_tfidf = get_CNN(tfidf, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='softmax')
cnn_w2v = get_CNN(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True)
cnn_glove = get_CNN(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True)
cnn_fast = get_CNN(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          4377000   
_________________________________________________________________
dropout (Dropout)            (None, 300, 300)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 298, 64)           57664     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation (Activation)      (None, 256)               0

In [39]:
cnn_cv.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=20)

Epoch 1/20
677/677 [==============================] - 60s 87ms/step - loss: 0.8142 - accuracy: 0.6845 - val_loss: 0.7720 - val_accuracy: 0.6909
Epoch 2/20
677/677 [==============================] - 59s 87ms/step - loss: 0.7128 - accuracy: 0.7298 - val_loss: 0.6511 - val_accuracy: 0.7726
Epoch 3/20
677/677 [==============================] - 59s 87ms/step - loss: 0.6133 - accuracy: 0.7740 - val_loss: 0.5767 - val_accuracy: 0.7768
Epoch 4/20
677/677 [==============================] - 59s 87ms/step - loss: 0.5667 - accuracy: 0.7865 - val_loss: 0.5534 - val_accuracy: 0.7831
Epoch 5/20
677/677 [==============================] - 59s 87ms/step - loss: 0.5384 - accuracy: 0.7938 - val_loss: 0.5170 - val_accuracy: 0.8050
Epoch 6/20
677/677 [==============================] - 59s 87ms/step - loss: 0.5139 - accuracy: 0.8025 - val_loss: 0.5180 - val_accuracy: 0.7970
Epoch 7/20
677/677 [==============================] - 59s 87ms/step - loss: 0.4906 - accuracy: 0.8126 - val_loss: 0.4798 - val_accuracy:

In [40]:
cnn_tfidf.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_w2v.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_glove.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_fast.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)

Epoch 1/20
68/68 [==============================] - 51s 738ms/step - loss: 0.8569 - accuracy: 0.6827 - val_loss: 0.8189 - val_accuracy: 0.6850
Epoch 2/20
68/68 [==============================] - 50s 734ms/step - loss: 0.8098 - accuracy: 0.6850 - val_loss: 0.7955 - val_accuracy: 0.6850
Epoch 3/20
68/68 [==============================] - 50s 736ms/step - loss: 0.7811 - accuracy: 0.6850 - val_loss: 0.7693 - val_accuracy: 0.6875
Epoch 4/20
68/68 [==============================] - 50s 736ms/step - loss: 0.7366 - accuracy: 0.7075 - val_loss: 0.7065 - val_accuracy: 0.7357
Epoch 5/20
68/68 [==============================] - 50s 735ms/step - loss: 0.6895 - accuracy: 0.7468 - val_loss: 0.6622 - val_accuracy: 0.7655
Epoch 6/20
68/68 [==============================] - 50s 736ms/step - loss: 0.6443 - accuracy: 0.7663 - val_loss: 0.6151 - val_accuracy: 0.7764
Epoch 7/20
68/68 [==============================] - 50s 737ms/step - loss: 0.6123 - accuracy: 0.7767 - val_loss: 0.5863 - val_accuracy: 0.7814

In [41]:
# print(df_prob_train['CNN-CV-0'])
# # for ext in ['CV', 'TFIDF', 'W2V', 'GLOVE', 'FAST']:
# #   for i in range(0,3):
# #     df_prob_train = df_prob_train.drop(columns=['CNN-{}-{}'.format(ext, i)])
# #     df_prob_val = df_prob_val.drop(columns=['CNN-{}-{}'.format(ext, i)])
# #     df_prob_test = df_prob_test.drop(columns=['CNN-{}-{}'.format(ext, i)])

In [42]:
print(df_prob_train.columns)

Index(['norm', 'SVM-CV-0', 'SVM-CV-1', 'SVM-CV-2', 'SVM-TFIDF-0',
       'SVM-TFIDF-1', 'SVM-TFIDF-2', 'SVM-W2V-0', 'SVM-W2V-1', 'SVM-W2V-2',
       ...
       'KNN-TFIDF-2', 'KNN-W2V-0', 'KNN-W2V-1', 'KNN-W2V-2', 'KNN-GLOVE-0',
       'KNN-GLOVE-1', 'KNN-GLOVE-2', 'KNN-FAST-0', 'KNN-FAST-1', 'KNN-FAST-2'],
      dtype='object', length=106)


In [43]:
# df_pred_test = df_pred_test.drop(['CNN-CV'], axis=1)
# print(df_pred_test.columns)
# CV
cols = ["CNN-CV-0", "CNN-CV-1", "CNN-CV-2"]
df_train_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_train), axis=1), columns=["CNN-CV"])
df_val_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_val), axis=1), columns=["CNN-CV"])
df_test_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_test), axis=1), columns=["CNN-CV"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# print(df_pred_test.columns)

# # Probabilidades
  
df_train_ = pd.DataFrame(cnn_cv.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_cv.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_cv.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-TF-0", "CNN-TF-1", "CNN-TF-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_train), axis=1), columns=["CNN-TFIDF"])
df_val_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_val), axis=1), columns=["CNN-TFIDF"])
df_test_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_test), axis=1), columns=["CNN-TFIDF"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_tfidf.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_tfidf.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_tfidf.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-W2V-0", "CNN-W2V-1", "CNN-W2V-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_train), axis=1), columns=["CNN-W2V"])
df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_w2v.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-GLOVE-0", "CNN-GLOVE-1", "CNN-GLOVE-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_train), axis=1), columns=["CNN-GLOVE"])
df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_glove.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-FAST-0", "CNN-FAST-1", "CNN-FAST-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_train), axis=1), columns=["CNN-FAST"])
df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_fast.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [ ]:
df_prob_train['norm']

In [ ]:
df_pred_train.to_csv("pred_train_zw.csv")
df_pred_val.to_csv("pred_val_zw.csv")
df_pred_test.to_csv("pred_test_zw.csv")

In [ ]:
df_prob_train.to_csv("prob_train_zw.csv")
df_prob_val.to_csv("prob_val_zw.csv")
df_prob_test.to_csv("prob_test_zw.csv")

In [47]:
df_pred_val.shape

(13525, 41)

(13525, 41)